In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# TechKey Analysis - Data Analysis Notebook\n",
    "\n",
    "This notebook provides exploratory data analysis and visualization for the student performance data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Import required libraries\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import plotly.express as px\n",
    "import plotly.graph_objects as go\n",
    "from plotly.subplots import make_subplots\n",
    "import sys\n",
    "import os\n",
    "\n",
    "# Add src to path\n",
    "sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd())))\n",
    "\n",
    "# Set plotting style\n",
    "plt.style.use('seaborn-v0_8')\n",
    "sns.set_palette(\"husl\")\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load data from database\n",
    "from src.database import get_session\n",
    "from src.models import Student, Grade, Attendance, Prediction\n",
    "\n",
    "session = get_session()\n",
    "\n",
    "# Load students with their metrics\n",
    "students = session.query(Student).all()\n",
    "\n",
    "# Create DataFrame for analysis\n",
    "student_data = []\n",
    "for student in students:\n",
    "    student_data.append({\n",
    "        'student_id': student.student_id,\n",
    "        'name': student.name,\n",
    "        'grade_avg': student.get_grade_average(),\n",
    "        'attendance_rate': student.get_attendance_rate(),\n",
    "        'course_count': len(student.enrollments),\n",
    "        'risk_level': student.get_risk_level()\n",
    "    })\n",
    "\n",
    "df = pd.DataFrame(student_data)\n",
    "print(f\"Loaded data for {len(df)} students\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Basic statistics\n",
    "print(\"=== Basic Statistics ===\")\n",
    "print(f\"Total students: {len(df)}\")\n",
    "print(f\"Average grade: {df['grade_avg'].mean():.2f}%\")\n",
    "print(f\"Average attendance: {df['attendance_rate'].mean():.2f}%\")\n",
    "print(f\"Average courses per student: {df['course_count'].mean():.2f}\")\n",
    "\n",
    "print(\"\\n=== Risk Level Distribution ===\")\n",
    "risk_counts = df['risk_level'].value_counts()\n",
    "for risk, count in risk_counts.items():\n",
    "    print(f\"{risk}: {count} students ({count/len(df)*100:.1f}%)\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Distribution plots\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "\n",
    "# Grade distribution\n",
    "axes[0, 0].hist(df['grade_avg'].dropna(), bins=20, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "axes[0, 0].set_title('Grade Distribution')\n",
    "axes[0, 0].set_xlabel('Grade Average (%)')\n",
    "axes[0, 0].set_ylabel('Number of Students')\n",
    "axes[0, 0].axvline(df['grade_avg'].mean(), color='red', linestyle='--', label=f'Mean: {df[\"grade_avg\"].mean():.1f}%')\n",
    "axes[0, 0].legend()\n",
    "\n",
    "# Attendance distribution\n",
    "axes[0, 1].hist(df['attendance_rate'].dropna(), bins=20, alpha=0.7, color='lightgreen', edgecolor='black')\n",
    "axes[0, 1].set_title('Attendance Distribution')\n",
    "axes[0, 1].set_xlabel('Attendance Rate (%)')\n",
    "axes[0, 1].set_ylabel('Number of Students')\n",
    "axes[0, 1].axvline(df['attendance_rate'].mean(), color='red', linestyle='--', label=f'Mean: {df[\"attendance_rate\"].mean():.1f}%')\n",
    "axes[0, 1].legend()\n",
    "\n",
    "# Course count distribution\n",
    "course_counts = df['course_count'].value_counts().sort_index()\n",
    "axes[1, 0].bar(course_counts.index, course_counts.values, color='orange', alpha=0.7)\n",
    "axes[1, 0].set_title('Course Enrollment Distribution')\n",
    "axes[1, 0].set_xlabel('Number of Courses')\n",
    "axes[1, 0].set_ylabel('Number of Students')\n",
    "\n",
    "# Risk level distribution\n",
    "risk_counts = df['risk_level'].value_counts()\n",
    "colors = ['#FF4444', '#FFAA44', '#44FF44']  # Red, Yellow, Green\n",
    "axes[1, 1].pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%', colors=colors)\n",
    "axes[1, 1].set_title('Risk Level Distribution')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Interactive scatter plot: Attendance vs Performance\n",
    "fig = px.scatter(df, \n",
    "                 x='attendance_rate', \n",
    "                 y='grade_avg',\n",
    "                 color='risk_level',\n",
    "                 hover_data=['student_id', 'name'],\n",
    "                 title='Attendance vs Performance by Risk Level',\n",
    "                 labels={\n",
    "                     'attendance_rate': 'Attendance Rate (%)',\n",
    "                     'grade_avg': 'Grade Average (%)',\n",
    "                     'risk_level': 'Risk Level'\n",
    "                 })\n",
    "\n",
    "# Add trend line\n",
    "z = np.polyfit(df['attendance_rate'], df['grade_avg'], 1)\n",
    "p = np.poly1d(z)\n",
    "x_range = np.linspace(df['attendance_rate'].min(), df['attendance_rate'].max(), 100)\n",
    "fig.add_trace(go.Scatter(x=x_range, y=p(x_range), \n",
    "                         mode='lines', \n",
    "                         name='Trend',\n",
    "                         line=dict(dash='dash', color='gray')))\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Correlation analysis\n",
    "numeric_cols = ['grade_avg', 'attendance_rate', 'course_count']\n",
    "correlation_matrix = df[numeric_cols].corr()\n",
    "\n",
    "fig = px.imshow(correlation_matrix,\n",
    "                title='Feature Correlation Matrix',\n",
    "                color_continuous_scale='RdBu_r',\n",
    "                aspect='auto')\n",
    "\n",
    "fig.show()\n",
    "\n",
    "print(\"Correlation Matrix:\")\n",
    "print(correlation_matrix)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Box plots by risk level\n",
    "fig = make_subplots(rows=1, cols=2, \n",
    "                    subplot_titles=('Grade Average by Risk Level', 'Attendance Rate by Risk Level'))\n",
    "\n",
    "# Grade average box plot\n",
    "for risk_level in df['risk_level'].unique():\n",
    "    risk_data = df[df['risk_level'] == risk_level]['grade_avg']\n",
    "    fig.add_trace(go.Box(y=risk_data, name=risk_level), row=1, col=1)\n",
    "\n",
    "# Attendance rate box plot\n",
    "for risk_level in df['risk_level'].unique():\n",
    "    risk_data = df[df['risk_level'] == risk_level]['attendance_rate']\n",
    "    fig.add_trace(go.Box(y=risk_data, name=risk_level), row=1, col=2)\n",
    "\n",
    "fig.update_layout(title='Performance Metrics by Risk Level',\n",
    "                  showlegend=False,\n",
    "                  height=500)\n",
    "\n",
    "fig.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Time series analysis (if date data is available)\n",
    "# Load grade history\n",
    "grades = session.query(Grade).all()\n",
    "\n",
    "if grades:\n",
    "    grade_data = []\n",
    "    for grade in grades:\n",
    "        grade_data.append({\n",
    "            'student_id': grade.student.student_id,\n",
    "            'assignment': grade.assignment_name,\n",
    "            'score': grade.score,\n",
    "            'date': grade.grade_date\n",
    "        })\n",
    "    \n",
    "    grades_df = pd.DataFrame(grade_data)\n",
    "    grades_df['date'] = pd.to_datetime(grades_df['date'])\n",
    "    \n",
    "    # Average grade over time\n",
    "    monthly_avg = grades_df.groupby(grades_df['date'].dt.to_period('M'))['score'].mean()\n",
    "    \n",
    "    fig = px.line(x=monthly_avg.index.astype(str), \n",
    "                  y=monthly_avg.values,\n",
    "                  title='Average Grade Over Time',\n",
    "                  labels={'x': 'Month', 'y': 'Average Grade (%)'})\n",
    "    \n",
    "    fig.show()\n",
    "else:\n",
    "    print(\"No grade history data available\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Predictive analytics insights\n",
    "predictions = session.query(Prediction).all()\n",
    "\n",
    "if predictions:\n",
    "    pred_data = []\n",
    "    for pred in predictions:\n",
    "        pred_data.append({\n",
    "            'student': pred.student.name,\n",
    "            'prediction': pred.prediction,\n",
    "            'probability': pred.probability,\n",
    "            'date': pred.prediction_date\n",
    "        })\n",
    "    \n",
    "    pred_df = pd.DataFrame(pred_data)\n",
    "    \n",
    "    # Model performance analysis\n",
    "    print(\"=== Prediction Analysis ===\")\n",
    "    print(f\"Total predictions: {len(pred_df)}\")\n",
    "    print(f\"High risk predictions: {len(pred_df[pred_df['prediction'] == 'High Risk'])}\")\n",
    "    print(f\"Average probability for high risk: {pred_df[pred_df['prediction'] == 'High Risk']['probability'].mean():.3f}\")\n",
    "    print(f\"Average probability for low risk: {pred_df[pred_df['prediction'] == 'Low Risk']['probability'].mean():.3f}\")\n",
    "    \n",
    "    # Probability distribution\n",
    "    fig = px.histogram(pred_df, \n",
    "                       x='probability', \n",
    "                       color='prediction',\n",
    "                       title='Prediction Probability Distribution',\n",
    "                       nbins=20)\n",
    "    fig.show()\n",
    "else:\n",
    "    print(\"No prediction data available\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Export summary statistics\n",
    "summary_stats = df.describe()\n",
    "print(\"=== Summary Statistics ===\")\n",
    "print(summary_stats)\n",
    "\n",
    "# Save analysis results\n",
    "analysis_results = {\n",
    "    'total_students': len(df),\n",
    "    'avg_grade': df['grade_avg'].mean(),\n",
    "    'avg_attendance': df['attendance_rate'].mean(),\n",
    "    'high_risk_count': len(df[df['risk_level'] == 'High Risk']),\n",
    "    'grade_attendance_correlation': df['grade_avg'].corr(df['attendance_rate'])\n",
    "}\n",
    "\n",
    "print(\"\\n=== Key Insights ===\")\n",
    "for key, value in analysis_results.items():\n",
    "    if 'correlation' in key:\n",
    "        print(f\"{key}: {value:.3f}\")\n",
    "    elif 'avg' in key:\n",
    "        print(f\"{key}: {value:.1f}%\")\n",
    "    else:\n",
    "        print(f\"{key}: {value}\")\n",
    "\n",
    "# Close database session\n",
    "session.close()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Key Findings\n",
    "\n",
    "Based on the analysis:\n",
    "\n",
    "1. **Grade Distribution**: Most students perform in the [range] with an average of [value]%\n",
    "2. **Attendance Impact**: Strong correlation between attendance and performance (r = [value])\n",
    "3. **Risk Factors**: [Number] students identified as high risk, primarily due to [reasons]\n",
    "4. **Course Load**: Average of [value] courses per student with [insight about distribution]\n",
    "\n",
    "## Recommendations\n",
    "\n",
    "1. **Targeted Interventions**: Focus on students with attendance below [threshold]%\n",
    "2. **Early Warning System**: Implement alerts for grade drops below [threshold]%\n",
    "3. **Academic Support**: Provide additional resources for high-risk students\n",
    "4. **Regular Monitoring**: Track performance trends over time for proactive support"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

: 